## Get most clusters with only one class 

- ordernar clusters por numero de pixeis (ordem decrescente);
- selecao de 30% dos pixeis, dando prioridade aos clusters que aparecem primeiro na lista

In [ ]:
train_f1 = r'G:\autocls_exp2\lcfilter\f1_lisboa.tif'

clusters = r'G:\autocls_exp2\cluster_filter\cmb_km_f1_lisboa.tif'

result = r'G:\autocls_exp2\lcfilter\f6_lisboa.tif'

In [ ]:
import os
import pandas as pd
import numpy as np
from glass.rst.local import combine
from glass.pys.oss import mkdir
from glass.pys.tm import now_as_str
from glass.rd import tbl_to_obj
from glass.esri.rst.local import combine_rst, lookup
from glass.wt import obj_to_tbl
from glass.esri.tbl.joins import join_table
from glass.esri.tbl.col import cols_calc
from glass.esri.rd.rst import rst_to_lyr
from glass.esri.rst.rcls import rcls_rst
from glass.esri.rd import dbf_to_view
from glass.dtt.mge.pd import merge_df

In [ ]:
tmpws = mkdir(
    os.path.join(os.path.dirname(result), now_as_str(utc=True)),
    overwrite=True
)

# Combine
clstbycls, clyr = combine_rst(
    [train_f1, clusters],
    os.path.join(tmpws, 'c6_cls_cluster.tif'),
    train_f1
)

vatdbf = os.path.join(tmpws, f"{os.path.basename(clstbycls)}.vat.dbf")
df_clstbycls = tbl_to_obj(vatdbf)

class_col = df_clstbycls.columns.values[2]
cluster_col = df_clstbycls.columns.values[3]

In [ ]:
df_clstbycls

In [ ]:
# Count number of classes for each cluster
clstsum = pd.DataFrame({
    'nclasses' : df_clstbycls.groupby([cluster_col])[class_col].agg('count')
}).reset_index()

clstsum['classv'] = np.where(
    clstsum.nclasses > 1,
    0, 1
)

# Save table as DBF
clstnum_dbf = obj_to_tbl(clstsum, os.path.join(tmpws, 'ncls_by_cluster.dbf'))

In [ ]:
clstsum.rename(columns={cluster_col: 'fk'}, inplace=True)

df_clstbycls = df_clstbycls.merge(clstsum, how='left', left_on=cluster_col, right_on='fk')

df_clstbycls = df_clstbycls[df_clstbycls.classv == 1]
df_clstbycls.reset_index(inplace=True)

In [ ]:
# Get px count of each class
class_df = pd.DataFrame({
    'npx' : df_clstbycls.groupby([class_col])['Count'].agg('sum')
}).reset_index()

In [ ]:
# Get relative freg
class_df.rename(columns={class_col: 'class_fk'}, inplace=True)

df_clstbycls  = df_clstbycls.merge(class_df, how='left', left_on=class_col, right_on='class_fk')

df_clstbycls['relfreq'] = df_clstbycls['Count'] / df_clstbycls['npx'] * 100

In [ ]:
# Get clusters with more pixels for each class
lclasses = class_df['class_fk'].tolist()

cdfs = []
for cls in lclasses:
    cdf = df_clstbycls[df_clstbycls[class_col] == cls]

    # Order By frequency | descending ordering
    cdf = cdf.sort_values(by='relfreq', ascending=False)

    # Get cumulative freq
    cdf['cumfreq'] = cdf['relfreq'].cumsum()

    # Get wanted clusters
    cdf = cdf[cdf.cumfreq <= 30]

    cdfs.append(cdf)

wclst = merge_df(cdfs)

In [ ]:
# Reclassify Combine raster
wclst['rcls'] = wclst[['Value', class_col]].values.tolist()

In [ ]:
rclsrules = wclst.rcls.tolist()

rcls_rst(clyr, 'Value', rclsrules, result, train_f1)